In [55]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [56]:
df=pd.read_csv('train_final1.csv')

In [57]:
# Créer une série temporaire avec prénoms normalisés
df_temp = df.copy()
df_temp['prenom'] = df_temp['prenom'].str.lower().str.strip()

# Convertir en dictionnaire
dictionnaire_prenoms = df_temp.set_index('prenom')['gender'].to_dict()
len(dictionnaire_prenoms)

627

In [58]:
# ========== 2. EXTRACTION DES FEATURES ==========
def extract_features(name):
    name = name.lower().strip()
    features = {
        'first_letter': name[0] if len(name) > 0 else '',
        'last_letter': name[-1] if len(name) > 0 else '',
        'first_2': name[:2] if len(name) >= 2 else name,
        'last_2': name[-2:] if len(name) >= 2 else name,
        'length': len(name)
    }
    return features

In [59]:
features_list = [extract_features(p) for p in df['prenom']]
df_features = pd.DataFrame(features_list)

# 3) encodage
encoders = {}
from sklearn.preprocessing import LabelEncoder
for col in ['first_letter', 'last_letter', 'first_2', 'last_2']:
    encoders[col] = LabelEncoder()
    df_features[col] = encoders[col].fit_transform(df_features[col])

X = df_features
y = df['gender'].map({'m': 0, 'f': 1})

mask = ~y.isna()
X = X[mask].reset_index(drop=True)
y = y[mask].reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4) entraînement du modèle
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_split=5, n_jobs=-1,
                       random_state=42)

In [60]:
def prediction(name):
    name_clean = name.lower().strip()
    
    # 1) Lookup
    if name_clean in dictionnaire_prenoms:
        gender_code = dictionnaire_prenoms[name_clean]
        gender = 'Masculin' if gender_code == 'm' else 'Féminin'
        return gender, 100.0, 'Lookup'
    
    # 2) Sinon : Random Forest
    else:
        # 2.1 features brutes
        features = extract_features(name_clean)  # dict
        
        # 2.2 encodage
        for col in ['first_letter', 'last_letter', 'first_2', 'last_2']:
            try:
                features[col] = encoders[col].transform([features[col]])[0]
            except:
                features[col] = -1   # valeur inconnue
        
        # 2.3 DataFrame 1 ligne
        X_new = pd.DataFrame([features])
        
        # 2.4 prédiction
        pred = model.predict(X_new)[0]
        proba = model.predict_proba(X_new)[0]
        
        gender = 'Masculin' if pred == 0 else 'Féminin'
        confidence = max(proba) * 100
        
        return gender, confidence, 'Random Forest'


In [61]:
print(prediction("Ahmed"))      # devrait renvoyer ('Masculin', 100.0, 'Lookup') si Ahmed est dans le CSV
print(prediction("  FATIMA "))  # ('Féminin', 100.0, 'Lookup')
print(prediction("Tariq"))      # ('Masculin' ou 'Féminin', X%, 'Random Forest')
print(prediction("Nour"))       # souvent 'Random Forest' aussi


('Masculin', 100.0, 'Lookup')
('Féminin', 100.0, 'Lookup')
('Masculin', 100.0, 'Lookup')
('Féminin', 100.0, 'Lookup')


In [62]:
df_test = pd.read_csv('test.csv')   # colonnes: ['nom','prenom']

# 2. Appliquer la fonction à chaque prénom
resultats = df_test['prenom'].apply(prediction)  # chaque élément = (genre, confiance, méthode)

# 3. Décomposer le tuple en 3 colonnes
df_test['genre_predit']   = resultats.apply(lambda x: x[0])
df_test['confiance']      = resultats.apply(lambda x: float(x[1]))
df_test['methode']        = resultats.apply(lambda x: x[2])

# 4. Afficher un aperçu
print(df_test.head(20))

# 5. Distribution par méthode
print("\nDistribution par méthode :")
print(df_test['methode'].value_counts())

# 6. Sauvegarder le résultat final
df_test.to_csv('predictions_hybride.csv', index=False)
print("\nFichier sauvegardé : predictions_hybride.csv")

            nom    prenom genre_predit   confiance        methode
0         HAFID     tariq     Masculin  100.000000         Lookup
1        CHERIF       ali     Masculin  100.000000         Lookup
2        MOUSSA     karim     Masculin  100.000000         Lookup
3       BELKADI   khadija      Féminin   91.343915  Random Forest
4          KACI      hani     Masculin  100.000000         Lookup
5        MOUSSA  mohammed     Masculin   75.152146  Random Forest
6     BENCHEMLA     karim     Masculin  100.000000         Lookup
7        MAHREZ     karim     Masculin  100.000000         Lookup
8   ALIMENTAIRE       spa      Féminin   57.677778  Random Forest
9       SLIMANI     layla      Féminin   99.749366  Random Forest
10       MAHREZ     kadir     Masculin  100.000000         Lookup
11        NOURI    samira      Féminin  100.000000         Lookup
12       SOUARI      hana      Féminin   89.445795  Random Forest
13        MEKLI   youssef     Masculin  100.000000         Lookup
14      SL

In [63]:
import pandas as pd

# 1) Charger train et test
train = pd.read_csv('train_final1.csv')       # contient: prenom, gender
test  = pd.read_csv('test.csv')             # contient: nom, prenom

# 2) Normaliser les prénoms
train_p = train['prenom'].str.lower().str.strip()
test_p  = test['prenom'].str.lower().str.strip()

# 3) Ensembles
set_train = set(train_p)
set_test  = set(test_p)

# 4) Prénoms présents dans le test mais absents du train
missing = sorted(list(set_test - set_train))

print(f"Prénoms dans train : {len(set_train)}")
print(f"Prénoms dans test  : {len(set_test)}")
print(f"Prénoms manquants  : {len(missing)}")
print(missing[:50])   # aperçu


Prénoms dans train : 627
Prénoms dans test  : 66
Prénoms manquants  : 16
['atelier', 'bureau', 'chloe', 'dina', 'hana', 'khadija', 'layla', 'mohammed', 'rana', 'sa', 'salma', 'spa', 'université', 'yasmine', 'établissement', 'étude']


In [64]:
rows = [
    ['tariq',  'm'],
    ['tareq',  'm'],
    ['tarek',  'm'],
    ['tarik',  'm'],
    
    ['hamza',  'm'],
    ['hamzah', 'm'],
    
    ['aicha',  'f'],
    ['aïcha',  'f'],
    ['aisha',  'f'],
    ['aischa', 'f'],
    
    ['youssef', 'm'],
    ['youcef',  'm'],
    ['yusuf',   'm'],
    
    ['sarah', 'f'],
    ['sara',  'f'],
    ['feriel',  'f'],
    ['cherin',  'f'],
    ['nermine',  'f'],
    ['nermin',  'f'],
    
    ['lina',  'f'],
    ['lyna',  'f'],
    ['mustapha',  'm'],
    ['mustafa',  'm'],
    ['moustafa', 'm'],
    ['fatah', 'm'],
    ['djelloul',  'm'],
    ['jelloul',  'm'],
]

df_var = pd.DataFrame(rows, columns=['prenom', 'gender'])
df_var.to_excel('lookup_variantes_prenoms.xlsx', index=False)


In [66]:
prediction("jelloul")

('Masculin', 100.0, 'Lookup')

In [67]:
import os
print(os.getcwd())


C:\Users\User\stage2.0
